In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#for debug
%load_ext autoreload
%autoreload 2
import statsmodels.api as sm
plt.style.use("C:/Users/Ysh/.matplotlib/Paper.mplstyle")

In [2]:
input_para=[1, 1, 'normal_ic', 'D']#
return_time = input_para[0]
factor_back_time = input_para[1]
ic_types=input_para[2]
factor_type=input_para[3]

df = pd.read_hdf('D:/OneDrive/test-LL/swhy/sw_work/data/ic_results_data/\
type_%s_factor_and_return_%s_min_back_%s_min_%s_2020-2021.h5'%(factor_type, str(return_time), \
                                                               factor_back_time, ic_types))
# print(df.tail(3))


In [2]:

def calculate_t_value():
    result_df = pd.DataFrame()
    df['TradingDate'] = df['TradingTime'].dt.date
    grouped_df = df.groupby(['TradingDate'])
    index = 0 
    for date, df_oneday in grouped_df: 
        index+=1
        # if index > 10:
        #     break
        columns_list = df_oneday.columns.drop(['TradingDate','Symbol','TradingTime','return'])
        # df_oneday = df_oneday.fillna(method='ffill')
        
        result = pd.DataFrame(index=[date])
        result['Symbol'] = [df_oneday['Symbol'].iloc[0]]
        result['TradingDate'] = [df_oneday['TradingTime'].dt.date.iloc[0]]
        for column in range(len(columns_list)):
            x = df_oneday[columns_list[column]]
            y = df_oneday['return']
            
            # delete nan factor number
            return_data = y[x.notna()]
            factor_one = x.dropna()
            # ic_value = [np.corrcoef(factor_one, return_data)[0, 1]]
            #线性回归
            factor_one=sm.add_constant(factor_one)
            model=sm.OLS(return_data, factor_one).fit()
            coef = model.params[1]
            t_value = model.tvalues[1]
            result[columns_list[column]] = t_value

        result_df = pd.concat([result_df, result]).reset_index(drop=True)
    
    result_df.to_csv("../plots/test_D_t_test.csv")               
calculate_t_value()


C:\Users\Ysh\AppData\Local\Temp\ipykernel_14676\3106077852.py:16: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for date, df_oneday in grouped_df:


In [3]:
t_test_df =  pd.read_csv("../plots/test_D_t_test.csv") 


input_para=[1, 1, 'normal_ic', 'D']#
return_time = input_para[0]
factor_back_time = input_para[1]
ic_types=input_para[2]
factor_type=input_para[3]

ic_result_df = pd.read_csv('D:/OneDrive/test-LL/swhy/sw_work/data/ic_results_data/\
type_%s_ic_results_return_%s_min_back_%s_min_%s_2020-2021.csv'%(factor_type, str(return_time), factor_back_time, ic_types))
print(ic_result_df.shape)


columns_list = t_test_df.columns.drop(['TradingDate','Symbol'])
from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('../plots/%s_IC_VS_t_test_with_time_multipage_plots.pdf'%factor_type) as pdf:
    for index in range(len(columns_list)):      
        # draw figures
        fig, ax = plt.subplots(figsize=(12, 7))
        ax2 = ax.twinx()
        ax.plot(t_test_df['TradingDate'], t_test_df[columns_list[index]],'r.',label="t-test mean:%0.4f"%t_test_df[columns_list[index]].mean())
        ax2.plot(ic_result_df['TradingDate'], ic_result_df[columns_list[index]],'b.', label="IC mean:%0.4f"%ic_result_df[columns_list[index]].mean())
        import matplotlib.dates as mdates
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=int(len(t_test_df)/7)))
        ax.xaxis.set_tick_params(rotation=30)
        
        ax.legend(frameon=True,loc='upper left')
        ax2.legend(frameon=True,loc='upper right')
        ax.set_ylabel("T-test value")
        ax2.set_ylabel("IC value")
        plt.title(columns_list[index])
        plt.tight_layout()
        pdf.savefig(fig)
        plt.close(fig) 


from result_ana import cal_t_teststa

cal_t_teststa(t_test_df, "t-test_D")

(486, 16)
